# Testando a API

No NoteBook abaixo irei apresentar um script para teste da API desenvolvida no projeto de previsão de vendas das lojas Rossmann

In [58]:
# Importando bibliotecas
import requests
import json
import pandas as pd

In [68]:
# Carregando os dados de teste em produção
test_df = pd.read_csv("test.csv")
store_df = pd.read_csv("store.csv")

# Juntando os dados das lojas com os dados das vendas
df = pd.merge(test_df, store_df, how = 'left', on = "Store")

# Teste da API

# Realizando a predição de lojas aleatório, basta escolher o valor da variável n_lojas e descomentar a linha abaixo:
n_lojas = 5
df = df[df["Store"].isin(df["Store"].sample(n_lojas))]

# Selecionando as lojas para as quais serão realizadas as predições, basta preencher o número das lojas na lista abaixo e descomentar a linha a seguir:
lojas = [22]
#df = df[df["Store"].isin(lojas)]

# Retirando do DF de teste os dias que as lojas estavam fechadas e dados faltantes
df = df[df["Open"] == 1]
df = df[~df["Open"].isnull()]

In [69]:
# Convertendo o DF em Json
data = json.dumps(df.to_dict(orient = "records"))

In [70]:
url = "http://192.168.15.37:5000/rossmann/predict"
header = {'Content-type': "application/json"}
data = data

r = requests.post(url, data = data, headers = header)
print("Status Code {}".format(r.status_code))

Status Code 200


In [71]:
df1 = pd.DataFrame(r.json(), columns = r.json()[0].keys())

In [72]:
for i in df1["store"].unique():
    loja = i
    receita = df1[df1["store"] == loja]["prediction"].sum()
    print("A loja {} irá gerar R$ {:,.2f} de receita nos próximos 6 meses".format(loja, receita))

A loja 74 irá gerar R$ 256,667.83 de receita nos próximos 6 meses
A loja 101 irá gerar R$ 316,273.33 de receita nos próximos 6 meses
A loja 298 irá gerar R$ 351,027.57 de receita nos próximos 6 meses
A loja 447 irá gerar R$ 215,054.70 de receita nos próximos 6 meses
A loja 831 irá gerar R$ 515,533.81 de receita nos próximos 6 meses
